In [1]:
import whisper
import torch
import os
os.listdir()

torch.cuda.is_available()
torch.cuda.empty_cache()


In [2]:
model = whisper.load_model("base.en")
audio = whisper.load_audio('test2.mp3')

result = model.transcribe(audio)
print(result["text"])

 I'm sorry, that was a restaurant thing. Oh, yeah, yeah, yeah. I was gonna just ask, like, who did you receive with? I'm sorry. I'm sorry, I'm sorry. Shon is funding education. Yeah. Okay, so lots of people reach out to her and then she gets a touch of me. They can flow reach out to her and stuff. The areas that are, they're seeing more like, people or like needles for you. Okay. What's that? She'll just, she'll reach out to me and then I'll go kind of to the area and keep on it for a little bit and see how it exists. How free is that? I was like, I wonder if it's like a citizen or if it's like a business area. That's like, I guess the two. Yeah. It's like coming to my community. Like, yes, it's like, pretty easy. Yeah, through the calls and how are we going to do this? How percity? It seems like a good mixture now that it's warm on us. Yeah, it's like, it's like, it's just really good. I don't know. It's really good. It's really good. Yeah, you and me. You just have it right now. Yeah

In [15]:
from ctransformers import AutoModelForCausalLM, AutoConfig

config = AutoConfig.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF")
# Explicitly set the max_seq_len
config.max_seq_len = 4096
config.max_answer_len= 4096

# Set gpu_layers to the number of layers to offload to GPU. 
# Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=100,
    hf=True,
    config=config,
    device_map="auto"
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# from transformers import AutoModelForCausalLM

# # Set gpu_layers to the number of layers to offload to GPU. 
# # Set to 0 if no GPU acceleration is available on your system.
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
#                                              gpu_layers=50)

In [25]:
from transformers import AutoTokenizer, pipeline
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
torch.cuda.empty_cache()

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Pipeline
generator = HuggingFacePipeline(
    pipeline=
        pipeline(
            model=model, 
            tokenizer=tokenizer, 
            config=config,
            task='text-generation',
            max_new_tokens = 100,
            device_map='auto'),
    batch_size=10)


In [19]:
#Verifying LLM
response = generator("What is 1+1?")
response
print(response)


A: 2


In [22]:
#Truncating length
print(len(result["text"]))

if len(result["text"]) / 4 > 256:
    print("too long")
    result["text"] = result["text"][1:512]
else:
    print("OK!")

print(len(result["text"]))
print(result["text"])

512
OK!
512
 I'm sorry, that was a restaurant thing. Oh, yeah, yeah, yeah. I was gonna just ask, like, who did you receive with? I'm sorry. I'm sorry, I'm sorry. Shon is funding education. Yeah. Okay, so lots of people reach out to her and then she gets a touch of me. They can flow reach out to her and stuff. The areas that are, they're seeing more like, people or like needles for you. Okay. What's that? She'll just, she'll reach out to me and then I'll go kind of to the area and keep on it for a little bit and see how


In [27]:
#Prompting
prompt = f"""
Return any short phrases (4-5 words) that refer to a location or place from the following document in a bulleted list. Do not return anything else as part of your answer.
Here is the following document:
\"{result["text"]}\"


"""
response = generator(prompt)
print(response)

* Restaurant
* Education
* Shon
* Touch
* Flow
* Area
* Needles
* Reach out
* Go
* Keep on it
* Little bit
* See how
* Me
* Kind of
* Touch
* Area
* Needles
* Reach out
* Go
* Keep on it
* Little bit
* See how
* Me
* Kind of
* Touch
* Area
* Needles

